<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Feature selection from 3 tools
### SGDClassifier, DecisionTree, LinearRegressor

### get most important 10 features for the next analysis,
###  result data (accuracy, precision, confusion matrix etc.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

In [10]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [11]:
import pickle
def list_to_pickle(filename, listname):
    open_file = open(filename, "wb")
    pickle.dump(listname, open_file)
    open_file.close()

def list_from_pickle(filename):
    open_file = open(filename, "rb")
    loaded_list = pickle.load(open_file)
    open_file.close()
    return loaded_list

In [12]:
# confusion matrix to list 변환
def matrix_to_list(confu_matrix):
    m_list = []
    tn = confu_matrix[0,0]
    fp = confu_matrix[0,1]
    fn = confu_matrix[1,0]
    tp = confu_matrix[1,1]
    m_list.extend([tn, fp, fn, tp])
    return m_list

In [13]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
#     cm = matrix_to_list(confusion_matrix(test_target, y_predict_list))
    collect_list = [ps, rs, fs, roc]
#     collect_list.extend(cm)
    return collect_list

In [14]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng'], '030200' : ['KT', 'kt']}

# code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy']}

In [15]:
# 분석용 데이터 입력
directory_for_ml = '../data/data_for_ml/expand_date/'

# 초기화
lr_accuracy = []
lr_test = []
lr_confu_matrix = []
sc_accuracy = []
sc_test = []
sc_confu_matrix = []
dt_accuracy = []
dt_test = []
dt_confu_matrix = []

for key, val in code.items():
    fname = f'df_{val[1]}_sel.pkl'
    f_name = directory_for_ml + fname
    df = pd.read_pickle(f_name) 
    
    # train, val,: 8, test: 2
    split_ratio = 0.8
    split_n = int(len(df)*split_ratio)

    data = df.iloc[:split_n, :-5]
    target = df.iloc[:split_n, -5]
    test_input = df.iloc[split_n:, :-5]
    test_target = df.iloc[split_n:, -5]
    
    train_input, val_input, train_target, val_target = train_test_split(data, target, 
                                                                        random_state=42, test_size=0.2, stratify=target)

    ss = StandardScaler()
    ss.fit(train_input)
    train_scaled = ss.transform(train_input)
    val_scaled = ss.transform(val_input)
    test_scaled = ss.transform(test_input)

    # ********** logistic regressor를 이용한 feature selection

    lr = LogisticRegression(C=20, max_iter=1000) # max_iter default 100, 
    #     lr = LogisticRegression(C=1, solver='newton_cg', max_iter=1000) # max_iter default 100, 
    lr.fit(train_scaled, train_target)

    train_score_lr = lr.score(train_scaled, train_target)
    val_score_lr = lr.score(val_scaled, val_target)
    test_score_lr = lr.score(test_scaled, test_target)
    
    lr_accuracy.append([val[1], train_score_lr, val_score_lr, test_score_lr])
    lr_test.append([val[1]] + predict_p(test_target, lr.predict(test_scaled)))
    cm = matrix_to_list(confusion_matrix(test_target, lr.predict(test_scaled)))
    lr_confu_matrix.append([val[1]] + cm)

    df_sel_lr = pd.DataFrame(lr.coef_[0, :], index=data.columns, columns=['importance_LR'])
    df_sel_lr['importance_LR'] = df_sel_lr['importance_LR'].apply(lambda x: abs(x))
    df_sel_lr = df_sel_lr.sort_values(by='importance_LR', ascending=False)
    # df_sel_lr.index
    

    # **********  결정트리를 사용하여 feature selection하기, feature_importances_ 이용

    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(train_input, train_target)

    train_score_dt = dt.score(train_input, train_target)
    val_score_dt = dt.score(val_input, val_target)
    test_score_dt = dt.score(test_input, test_target)
    
    dt_accuracy.append([val[1], train_score_dt, val_score_dt, test_score_dt])
    dt_test.append([val[1]] + predict_p(test_target, dt.predict(test_input)))
    cm = matrix_to_list(confusion_matrix(test_target, dt.predict(test_input)))
    dt_confu_matrix.append([val[1]] + cm)

    df_sel_dt = pd.DataFrame(dt.feature_importances_, index=data.columns, columns=['importance']).sort_values(by='importance', ascending=False)
    # df_sel_dt.index
    
 

    # **********  SGDClassifier를 이용한 feature selecition

    sc =  SGDClassifier(loss='log_loss', max_iter=2000, random_state=42)
    sc.fit(train_scaled, train_target)

    train_score_sc = sc.score(train_scaled, train_target)
    val_score_sc = sc.score(val_scaled, val_target)
    test_score_sc = sc.score(test_scaled, test_target)
    
    sc_accuracy.append([val[1], train_score_sc, val_score_sc, test_score_sc])
    sc_test.append([val[1]] + predict_p(test_target, sc.predict(test_scaled)))
    cm = matrix_to_list(confusion_matrix(test_target, sc.predict(test_scaled)))
    sc_confu_matrix.append([val[1]] + cm)

    df_sel_sc = pd.DataFrame(sc.coef_[0, :], index=data.columns, columns=['importance_SC'])
    df_sel_sc['importance_SC'] = df_sel_sc['importance_SC'].apply(lambda x: abs(x))
    df_sel_sc = df_sel_sc.sort_values(by='importance_SC', ascending=False)
    # df_sel_sc.index

    # ********** feature selection 하기
    select = {}
    for name in data.columns:
        sum = 0
        sum = (list(df_sel_sc.index).index(name) + 1)*2  \
             +(list(df_sel_lr.index).index(name) + 1)*2  \
             +(list(df_sel_dt.index).index(name) + 1)*1
        # 1부터 시작으로 하고 dt에는 1/2배 가중치줌. 왜?
        select[name] = sum

    # ------ top 10 important list  -----------------
    sorted_sel = sorted(select.items(), key = lambda items : items[1])
    new_columns = np.array(sorted_sel[:10])[:, 0]
    # -----------------------------------------------
 
    # logisticregression 결과 모으기

    df_lr_acc = pd.DataFrame(lr_accuracy, columns=['name', 'lr_train', 'lr_val', 'lr_test']).set_index('name')
    df_sc_acc = pd.DataFrame(sc_accuracy, columns=['name', 'sc_train', 'sc_val', 'sc_test']).set_index('name')
    df_dt_acc = pd.DataFrame(dt_accuracy, columns=['name', 'dt_train', 'dt_val', 'dt_test']).set_index('name')
    
    df_lr_test = pd.DataFrame(lr_test, columns=['name', 'lr_pre', 'lr_recall', 'lr_f1', 'lr_roc']).set_index('name')
    df_sc_test = pd.DataFrame(sc_test, columns=['name', 'sc_pre', 'sc_recall', 'sc_f1', 'sc_roc']).set_index('name')
    df_dt_test = pd.DataFrame(dt_test, columns=['name', 'dt_pre', 'dt_recall', 'dt_f1', 'dt_roc']).set_index('name')
        
    df_lr_cm = pd.DataFrame(lr_confu_matrix, columns=['name', 'lr_tn', 'lr_fp', 'lr_fn', 'lr_tp']).set_index('name')
    df_sc_cm = pd.DataFrame(sc_confu_matrix, columns=['name', 'sc_tn', 'sc_fp', 'sc_fn', 'sc_tp']).set_index('name')
    df_dt_cm = pd.DataFrame(dt_confu_matrix, columns=['name', 'dt_tn', 'dt_fp', 'dt_fn', 'dt_tp']).set_index('name')
    
    dfs = [df_lr_acc, df_sc_acc, df_dt_acc, df_lr_test, df_sc_test, df_dt_test, df_lr_cm, df_sc_cm, df_dt_cm ]
    df_merged = reduce(lambda  left,right: pd.merge(left,right, how='left', left_index=True, right_index=True), dfs)
    
    lr_col = [ x for x in df_merged.columns if x.startswith('lr')]
    sc_col = [ x for x in df_merged.columns if x.startswith('sc')]
    dt_col = [ x for x in df_merged.columns if x.startswith('dt')]

    df_lr = df_merged[lr_col]
    df_sc = df_merged[sc_col]
    df_dt = df_merged[dt_col]

In [16]:
acc_train = 0.
acc_val = 0.
acc_test = 0.
precision = 0.
# acc_val = 0.6
# acc_test = 0.6
# precision = 0.6
f1_score = 0.
recall = 0.

dt_ratio = ((df_merged['dt_fn'] + df_merged['dt_tp']) / (df_merged['dt_tn'] + df_merged['dt_fp'] + df_merged['dt_fn'] + df_merged['dt_tp']))
dt_sel = (df_merged['dt_train'] >= acc_train) & \
        (df_merged['dt_val'] >= acc_val) & \
        (df_merged['dt_test'] >= acc_test) & \
        (df_merged['dt_pre'] >= precision) & \
        (df_merged['dt_recall'] >= recall) & \
        (df_merged['dt_f1'] >= f1_score) 

sc_ratio = ((df_merged['sc_fn'] + df_merged['sc_tp']) / (df_merged['sc_tn'] + df_merged['sc_fp'] + df_merged['sc_fn'] + df_merged['sc_tp']))
sc_sel = (df_merged['sc_train'] >= acc_train) & \
        (df_merged['sc_val'] >= acc_val) & \
        (df_merged['sc_test'] >= acc_test) & \
        (df_merged['sc_pre'] >= precision) & \
        (df_merged['sc_recall'] >= recall) & \
        (df_merged['sc_f1'] >= f1_score) 

lr_ratio = ((df_merged['lr_fn'] + df_merged['lr_tp']) / (df_merged['lr_tn'] + df_merged['lr_fp'] + df_merged['lr_fn'] + df_merged['lr_tp']))
lr_sel = (df_merged['lr_train'] >= acc_train) & \
        (df_merged['lr_val'] >= acc_val) & \
        (df_merged['lr_test'] >= acc_test) & \
        (df_merged['lr_pre'] >= precision) & \
        (df_merged['lr_recall'] >= recall) & \
        (df_merged['lr_f1'] >= f1_score)

df_sel = df_merged[sc_sel & dt_sel & lr_sel]



In [17]:
df_sel[dt_col]

,dt_train,dt_val,dt_test,dt_pre,dt_recall,dt_f1,dt_roc,dt_tn,dt_fp,dt_fn,dt_tp
name,,,,,,,,,,,
sec,1.0,0.488889,0.535714,0.642857,0.529412,0.580645,0.537433,12,10,16,18
lgenergy,1.0,0.675676,0.521739,0.636364,0.500000,0.560000,0.527778,10,8,14,14
skhinix,1.0,0.571429,0.566038,0.500000,0.608696,0.549020,0.571014,16,14,9,14
ssbio,1.0,0.407407,0.545455,0.478261,0.785714,0.594595,0.577068,7,12,3,11
sdi,1.0,0.615385,0.693878,0.730769,0.703704,0.716981,0.692761,15,7,8,19
lgchemical,1.0,0.526316,0.574468,0.600000,0.500000,0.545455,0.576087,15,8,12,12
secpre,1.0,0.636364,0.642857,0.833333,0.555556,0.666667,0.677778,4,1,4,5
hyunmotor,1.0,0.526316,0.659574,0.722222,0.541667,0.619048,0.662138,18,5,11,13
naver,1.0,0.500000,0.531915,0.562500,0.375000,0.450000,0.535326,16,7,15,9


In [18]:
df_sel

,lr_train,lr_val,lr_test,sc_train,sc_val,sc_test,dt_train,dt_val,dt_test,lr_pre,...,lr_fn,lr_tp,sc_tn,sc_fp,sc_fn,sc_tp,dt_tn,dt_fp,dt_fn,dt_tp
name,,,,,,,,,,,,,,,,,,,,,
sec,0.921348,0.577778,0.553571,0.775281,0.600000,0.589286,1.0,0.488889,0.535714,0.680000,...,17,17,14,8,15,19,12,10,16,18
lgenergy,1.000000,0.513514,0.521739,0.931507,0.621622,0.456522,1.0,0.675676,0.521739,0.607143,...,11,17,5,13,12,16,10,8,14,14
skhinix,0.934132,0.642857,0.566038,0.742515,0.523810,0.566038,1.0,0.571429,0.566038,0.500000,...,10,13,20,10,13,10,16,14,9,14
ssbio,1.000000,0.592593,0.606061,0.961905,0.555556,0.666667,1.0,0.407407,0.545455,0.523810,...,3,11,12,7,4,10,7,12,3,11
sdi,1.000000,0.461538,0.551020,0.832258,0.461538,0.551020,1.0,0.615385,0.693878,0.600000,...,12,15,9,13,9,18,15,7,8,19
lgchemical,0.966216,0.552632,0.574468,0.804054,0.684211,0.617021,1.0,0.526316,0.574468,0.571429,...,8,16,14,9,9,15,15,8,12,12
secpre,1.000000,0.636364,0.500000,1.000000,0.636364,0.571429,1.0,0.636364,0.642857,0.750000,...,6,3,3,2,4,5,4,1,4,5
hyunmotor,1.000000,0.657895,0.510638,0.912752,0.605263,0.574468,1.0,0.526316,0.659574,0.520000,...,11,13,12,11,9,15,18,5,11,13
naver,1.000000,0.552632,0.553191,0.865772,0.631579,0.638298,1.0,0.500000,0.531915,0.578947,...,13,11,18,5,12,12,16,7,15,9


In [19]:
df_sel[lr_col]

,lr_train,lr_val,lr_test,lr_pre,lr_recall,lr_f1,lr_roc,lr_tn,lr_fp,lr_fn,lr_tp
name,,,,,,,,,,,
sec,0.921348,0.577778,0.553571,0.680000,0.500000,0.576271,0.568182,14,8,17,17
lgenergy,1.000000,0.513514,0.521739,0.607143,0.607143,0.607143,0.498016,7,11,11,17
skhinix,0.934132,0.642857,0.566038,0.500000,0.565217,0.530612,0.565942,17,13,10,13
ssbio,1.000000,0.592593,0.606061,0.523810,0.785714,0.628571,0.629699,9,10,3,11
sdi,1.000000,0.461538,0.551020,0.600000,0.555556,0.576923,0.550505,12,10,12,15
lgchemical,0.966216,0.552632,0.574468,0.571429,0.666667,0.615385,0.572464,11,12,8,16
secpre,1.000000,0.636364,0.500000,0.750000,0.333333,0.461538,0.566667,4,1,6,3
hyunmotor,1.000000,0.657895,0.510638,0.520000,0.541667,0.530612,0.509964,11,12,11,13
naver,1.000000,0.552632,0.553191,0.578947,0.458333,0.511628,0.555254,15,8,13,11


In [20]:
df_sel[dt_col]

,dt_train,dt_val,dt_test,dt_pre,dt_recall,dt_f1,dt_roc,dt_tn,dt_fp,dt_fn,dt_tp
name,,,,,,,,,,,
sec,1.0,0.488889,0.535714,0.642857,0.529412,0.580645,0.537433,12,10,16,18
lgenergy,1.0,0.675676,0.521739,0.636364,0.500000,0.560000,0.527778,10,8,14,14
skhinix,1.0,0.571429,0.566038,0.500000,0.608696,0.549020,0.571014,16,14,9,14
ssbio,1.0,0.407407,0.545455,0.478261,0.785714,0.594595,0.577068,7,12,3,11
sdi,1.0,0.615385,0.693878,0.730769,0.703704,0.716981,0.692761,15,7,8,19
lgchemical,1.0,0.526316,0.574468,0.600000,0.500000,0.545455,0.576087,15,8,12,12
secpre,1.0,0.636364,0.642857,0.833333,0.555556,0.666667,0.677778,4,1,4,5
hyunmotor,1.0,0.526316,0.659574,0.722222,0.541667,0.619048,0.662138,18,5,11,13
naver,1.0,0.500000,0.531915,0.562500,0.375000,0.450000,0.535326,16,7,15,9


In [13]:
df_sel[sc_col]

,sc_train,sc_val,sc_test,sc_pre,sc_recall,sc_f1,sc_roc,sc_tn,sc_fp,sc_fn,sc_tp
name,,,,,,,,,,,
sec,0.831395,0.697674,0.537037,0.592593,0.533333,0.561404,0.537500,13,11,14,16
lgenergy,0.798561,0.571429,0.659091,0.772727,0.629630,0.693878,0.667756,12,5,10,17
skhinix,0.737500,0.600000,0.509804,0.450000,0.391304,0.418605,0.499224,17,11,14,9
ssbio,0.852941,0.461538,0.562500,0.500000,0.642857,0.562500,0.571429,9,9,5,9
sdi,0.746667,0.526316,0.702128,0.727273,0.666667,0.695652,0.702899,17,6,8,16
lgchemical,0.736111,0.500000,0.555556,0.523810,0.523810,0.523810,0.553571,14,10,10,11
secpre,1.000000,0.700000,0.461538,0.666667,0.250000,0.363636,0.525000,4,1,6,2
hyunmotor,0.748252,0.583333,0.533333,0.500000,0.428571,0.461538,0.526786,15,9,12,9
naver,0.874126,0.555556,0.711111,0.750000,0.652174,0.697674,0.712451,17,5,8,15


In [ ]:
new_columns